Using [This link](https://github.com/cloudtostreet/MODIS_GlobalFloodDatabase).

Tellman et al, Satellite imaging reveals increased proportion of population exposed to floods; Nature; https://doi.org/10.1038/s41586-021-03695-w

Idai: 4725

main_floodmechanism.txt <br>
main_gfd.py

    from flood_detection import modis
    from flood_detection.utils import export, misc

    import time, os, csv

In [1]:
import ee
ee.Initialize()

In [2]:
import pandas as pd

df = pd.read_excel('/Users/alison/Documents/DPhil/global-flood-database/floodarchive_frombeth.xlsx')
coastal_causes = ["Storm", "Surge", "Hurricane", "Typhoon", "Cyclone"]
coastal_floods = df[df['MainCause'].apply(lambda cause: any(map(str(cause).__contains__, coastal_causes)))].copy()
coastal_floods['num_events'] = [1] * len(coastal_floods)
coastal_floods.groupby('Country')['num_events'].sum().sort_values(ascending=False)#[:10]
philippines = coastal_floods[coastal_floods['Country'] == "Philippines"]
philippines.head()

,ID,GlideNumber,Country,OtherCountry,long,lat,Area,Began,Ended,Validation,Dead,Displaced,MainCause,Severity,num_events
2251,2254,0,Philippines,0,125.009,11.7927,22834.67,2003-06-15,2003-06-18,News,11,70000,Tropical Cyclone,1.0,1
2282,2284,0,Philippines,0,121.267,16.5019,70175.04,2003-07-19,2003-07-23,News,22,20000,Tropical Cyclone,1.0,1
2860,2862,0,Philippines,0,124.860,11.2409,24930.10,2006-05-11,2006-05-14,News,41,42000,Tropical Cyclone,1.0,1
2919,2921,FL-2006-000098-PHI,Philippines,China,112.108,24.0035,612225.68,2006-07-11,2006-07-19,News,629,2450000,Tropical Cyclone,2.0,1
2930,2932,TC-2006-000105-CHN,Philippines,China,117.284,26.3940,350925.23,2006-07-24,2006-07-28,News,36,1500000,Tropical Cyclone,1.0,1


In [3]:
gfd = ee.ImageCollection('GLOBAL_FLOOD_DB/MODIS_EVENTS/V1')
gfdFloodedSum = gfd.select('flooded').sum()

if False:
    flood_maps = {}
    for flood in coastal_floods.itertuples():
        ID = flood.ID
        storm = storms = ee.ImageCollection(gfd.filterMetadata('id', 'equals', ID))
        flood_maps[ID] = storms.size().getInfo()

    flood_maps_df = pd.DataFrame.from_dict(flood_maps, orient='index').rename(columns={0: 'number_of_floods'})
    flood_maps_df = flood_maps_df.reset_index().rename(columns={'index': 'ID'})
    coastal_floods_present = flood_maps_df.merge(coastal_floods, on='ID', how='left')

    coastal_floods_present.to_csv('/Users/alison/Documents/DPhil/global-flood-database/floods_in_gfo.csv')
    
coastal_floods = pd.read_csv('/Users/alison/Documents/DPhil/global-flood-database/floods_in_gfo.csv', index_col=0)
coastal_floods = coastal_floods[coastal_floods['number_of_floods'] > 0].copy().reset_index(drop=True)
coastal_floods.head()

,ID,number_of_floods,GlideNumber,Country,OtherCountry,long,lat,Area,Began,Ended,Validation,Dead,Displaced,MainCause,Severity,num_events
0,2284,1,0,Philippines,0,121.267,16.5019,70175.04,2003-07-19,2003-07-23,News,22,20000,Tropical Cyclone,1.0,1
1,2348,1,0,South Korea,0,128.019,35.2349,21408.02,2003-09-12,2003-09-15,News,127,9000,Tropical Cyclone,1.0,1
2,2356,1,0,Mexico,0,-110.096,23.6058,113990.59,2003-09-22,2003-10-01,News,2,6000,Tropical Cyclone,1.0,1
3,2780,1,0,Australia,0,116.936,-23.2603,808059.61,2006-01-09,2006-01-31,News,0,1500,Tropical Cyclone,2.0,1
4,2815,1,0,Australia,0,114.989,-27.4100,37924.86,2006-03-03,2006-03-23,News,0,0,Tropical Cyclone,2.0,1


In [7]:
len(coastal_floods)

58

In [6]:
coastal_floods['Country'].value_counts()

Philippines    9
Mexico         7
USA            7
Australia      6
China          6
India          4
Cambodia       2
Madagascar     2
Vietnam        2
Thailand       2
Guatamala      1
South Korea    1
Bangladesh     1
New Zealand    1
Sri Lanka      1
Mozambique     1
North Korea    1
Oman           1
Burma          1
Bahamas        1
Pakistan       1
Name: Country, dtype: int64

    import pandas as pd

    keywords = ['tropical', 'cyclone', 'storm', 'typhoon', 'hurricane', 'surge']
    keywords = keywords + [keyword.capitalize() for keyword in keywords]

    events_table = pd.read_csv('gfd_event_stats_20215_13_error_fixed_2.csv')
    events_table = events_table[events_table['dfo_cause'].apply(lambda x: any(map(x.lower().__contains__, keywords)))]
    events_table['dfo_began'] = pd.to_datetime(events_table['dfo_began'])
    events_table = events_table[events_table['dfo_began'] >= pd.to_datetime('2017-01-01')]

    cols_of_interest = ['system:index', 'threshold_type','dfo_country',
                        'dfo_other_country', 'dfo_centroid_x', 'dfo_centroid_y',
                        'dfo_began', 'dfo_ended', 'dfo_validation',
                        'dfo_dead', 'dfo_displaced', 'dfo_cause', 
                        'dfo_severity', 'gfd_area']

    # manual
    index_dict = {4516: 'Hurricane Irma',
                 4676: 'Tropical Storm Florence',
                 4695: 'Hurricane Willa'}

    events_table = events_table.set_index('index')
    events_table[cols_of_interest]

In [29]:
[*coastal_floods.columns]

['ID',
 'number_of_floods',
 'GlideNumber',
 'Country',
 'OtherCountry',
 'long',
 'lat',
 'Area',
 'Began',
 'Ended',
 'Validation',
 'Dead',
 'Displaced',
 'MainCause',
 'Severity',
 'num_events']

{'ID': 2284,
 'number_of_floods': 1,
 'GlideNumber': '0',
 'Country': 'Philippines',
 'OtherCountry': '0',
 'long': 121.267,
 'lat': 16.5019,
 'Area': 70175.04,
 'Began': '2003-07-19',
 'Ended': '2003-07-23',
 'Validation': 'News',
 'Dead': 22,
 'Displaced': 20000,
 'MainCause': 'Tropical Cyclone',
 'Severity': 1.0,
 'num_events': 1}

In [35]:
idx = 0
metadata = coastal_floods.loc[idx, :].to_dict()

In [38]:
metadata

{'ID': 2284,
 'number_of_floods': 1,
 'GlideNumber': '0',
 'Country': 'Philippines',
 'OtherCountry': '0',
 'long': 121.267,
 'lat': 16.5019,
 'Area': 70175.04,
 'Began': '2003-07-19',
 'Ended': '2003-07-23',
 'Validation': 'News',
 'Dead': 22,
 'Displaced': 20000,
 'MainCause': 'Tropical Cyclone',
 'Severity': 1.0,
 'num_events': 1}

In [39]:
# load a flood
storms = ee.ImageCollection(
    gfd.filterMetadata('id', 'equals', metadata['ID']))

print(f"Number of flood maps: {storms.size().getInfo()}")
storm = ee.Image(storms.first())

# extract metadata
start = storm.get('began').getInfo()
end = storm.get('ended').getInfo()
cause = storm.get('dfo_main_cause').getInfo()
severity = storm.get('dfo_severity').getInfo()
country = storm.get('dfo_country').getInfo()

print(f"start date: {start}")
print(f"end date: {end}")
print(f"cause: {cause}")
print(f"severity: {severity}")
print(f"main country: {country}")

lat = storm.get('dfo_centroid_x').getInfo()
lon = storm.get('dfo_centroid_y').getInfo()

Number of flood maps: 1
start date: 2003-07-19
end date: 2003-07-23
cause: Tropical Cyclone
severity: 1
main country: Philippines


In [40]:
# have a look
import folium

def add_ee_layer(self, ee_image_object, vis_params, name):
    """Adds a method for displaying Earth Engine image tiles to folium map."""
    map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
    folium.raster_layers.TileLayer(
        tiles=map_id_dict['tile_fetcher'].url_format,
        attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
        name=name,
        overlay=True,
        control=True
    ).add_to(self)

# Add Earth Engine drawing method to folium.
folium.Map.add_ee_layer = add_ee_layer


durationPalette = ['C3EFFE', '1341E8', '051CB0', '001133']

m = folium.Map(location=[lon, lat], zoom_start=5)

# map extent
vizParams = {'min': 0, 'max': 1, 'palette': '001133'}
m.add_ee_layer(storm.select('flooded').selfMask(), vizParams, f"{stormDartmouthId} - Inundation Extent")

# durations
durationPalette = ['C3EFFE', '1341E8', '051CB0', '001133'];
m.add_ee_layer(storm.select('duration').selfMask(), {'min': 0, 'max': 4, 'palette': durationPalette},
  f"index_dict[stormDartmouthId] - Duration")

# all floods to view satellite-derived flood plain
m.add_ee_layer(gfdFloodedSum.selfMask(), {'min': 0, 'max': 10, 'palette': durationPalette}, 'GFD Satellite Observed Flood Plain');

# 
jrc = gfd.select('jrc_perm_water').sum().gte(1)
m.add_ee_layer(jrc.selfMask(), {'min': 0, 'max': 1, 'palette': 'C3EFFE'}, 'JRC Permanent Water')

m

In [113]:
# load coastline
# buffer coastline by 20km either direction
import geopandas as gpd

BUFFER = 50000 # meters


coastline = coastline.to_crs(3857).buffer(BUFFER)

/Users/alison/miniconda3/envs/hybridmodels/lib/python3.10/site-packages/pygeos/constructive.py:175: RuntimeWarning: invalid value encountered in buffer
  return lib.buffer(


In [ ]:
# GSSR: https://gee-community-catalog.org/projects/gssr/

In [118]:
# https://gee-community-catalog.org/projects/shoreline/ 
mainlands = ee.FeatureCollection('projects/sat-io/open-datasets/shoreline/mainlands');
# var big_islands = ee.FeatureCollection('projects/sat-io/open-datasets/shoreline/big_islands');
# var small_islands = ee.FeatureCollection('projects/sat-io/open-datasets/shoreline/small_islands');




In [120]:
# // Define a function to print metadata column names and datatypes. This function
# // is intended to be applied by the `evaluate` method which provides the
# // function a client-side dictionary allowing the 'columns' object of the
# // feature collection metadata to be subset by dot notation or bracket notation
# // (`tableMetadata['columns']`).
def getCols(tableMetadata):
    print(tableMetadata.columns)


# // Fetch collection metadata (`.limit(0)`) and apply the
# // previously defined function using `evaluate()`. The printed object is a
# // dictionary where keys are column names and values are datatypes.
mainlands.limit(0).evaluate(getCols);

AttributeError: 'FeatureCollection' object has no attribute 'evaluate'

In [126]:
mainlands.first().propertyNames().getInfo()

['OBJECTID',
 'Shape_Leng',
 'Area_km2',
 'Shape_Area',
 'FID_Contin',
 'FID_EApoly',
 'Coastline_',
 'system:index']

In [128]:

mainlands.first().get('FID_EApoly').getInfo()

-1

In [84]:
coastline_ee = ee.Geometry.Polygon(coastline.geometry.__geo_interface__['features'][0]['geometry']['coordinates'],
                                        proj=ee.Projection('EPSG:4326'))



In [102]:
import geemap

# get the projection and scale of the image
projection = storm.projection()
scale = projection.nominalScale()

clipped_flood = storm.select('flooded').clip(aoi_ee)
masked_flood = clipped_flood.eq(1)
polygon = masked_flood.reduceToVectors(scale=scale, geometry=masked_flood.geometry(), eightConnected=True)

# convert the polygon to a geodataframe
gdf = geemap.ee_to_geopandas(polygon)

# display the geodataframe
gdf.plot()

Exception: GeometryConstructors.Polygon: Inputs must be Number, List, or Geometry. Found type 'List<List<Float>>'.

In [109]:
geemap.ee_export_image(
    storm.select('flooded'), filename='test.tif', scale=90, region=aoi_ee, file_per_band=True
)

Generating URL ...
An error occurred while downloading.
GeometryConstructors.Polygon: Inputs must be Number, List, or Geometry. Found type 'List<List<Float>>'.


In [111]:
type(aoi_ee)

ee.geometry.Geometry

In [112]:
storm.select('flooded')

In [106]:
import geopandas as gpd
import ee
import geemap

ee.Initialize()

# load the image
image = storm.select('flooded')

# get the projection and scale of the image
projection = image.projection()
scale = projection.nominalScale()

# convert the image to a binary mask where 1 indicates the value you're interested in
mask = image.eq(1)

# convert the mask to a polygon
polygon = mask.reduceToVectors(scale=scale, geometry=mask.geometry(), eightConnected=True, bestEffort=True)

# convert the polygon to a geodataframe
gdf = geemap.ee_to_geopandas(polygon)

# display the geodataframe
gdf.plot()



Exception: Collection query aborted after accumulating over 5000 elements.

In [105]:
help(geemap.ee_to_geopandas)

Help on function ee_to_gdf in module geemap.common:

ee_to_gdf(ee_object, selectors=None, verbose=False)
    Converts an ee.FeatureCollection to Geopandas dataframe.
    
    Args:
        ee_object (ee.FeatureCollection): ee.FeatureCollection.
        selectors (list, optional): A list of attributes to export. Defaults to None.
        verbose (bool, optional): Whether to print out descriptive text. Defaults to False.
    
    Raises:
        TypeError: ee_object must be an ee.FeatureCollection.
    
    Returns:
        gpd.GeoDataFrame: geopandas.GeoDataFrame



In [95]:
mask = image.eq(1)

# convert the mask to a polygon
polygon = mask.reduceToVectors(scale=scale, geometry=mask.geometry(), eightConnected=True)

ee.featurecollection.FeatureCollection

In [91]:
flood_gdf = geemap.ee_to_geopandas(flood_vec)

Exception: GeometryConstructors.Polygon: Inputs must be Number, List, or Geometry. Found type 'List<List<Float>>'.

In [87]:
clipped_flood

Exception: Image.reduceToVectors: Too many pixels in the region. Found 17640622, but maxPixels allows only 10000000.
Ensure that you are not aggregating at a higher resolution than you intended; that is a frequent cause of this error. If not, then you may set the 'maxPixels' argument to a limit suitable for your computation; set 'bestEffort' to true to aggregate at whatever scale results in 'maxPixels' total pixels; or both.

# Export

In [34]:
# set up footprint
footprint = storm.get('system:footprint').getInfo()
footprint = ee.Geometry.LinearRing(footprint['coordinates'])

The main issue here is that we don't have a specific date. I am unsure how to resolve this to be compatible with the current form of the data.

In [36]:
band = 'duration'

task_config = {
    'scale': 30,  
    'region': footprint,
    'maxPixels': 5000000000
    }

task = ee.batch.Export.image(storm.select(band), f"{stormDartmouthId} - {band}", task_config)

task.start()

In [37]:
task.status()

{'state': 'READY',
 'description': '4217 - duration',
 'creation_timestamp_ms': 1682616152439,
 'update_timestamp_ms': 1682616152439,
 'start_timestamp_ms': 0,
 'task_type': 'EXPORT_IMAGE',
 'id': 'NM27PTW5C6GDD5SBS3FMJN4U',
 'name': 'projects/earthengine-legacy/operations/NM27PTW5C6GDD5SBS3FMJN4U'}

In [39]:
band = 'jrc_permwater'

task_config = {
    'scale': 30,  
    'region': footprint,
    'maxPixels': 5000000000
    }

task1 = ee.batch.Export.image(storm.select(band), f"{stormDartmouthId} - {band}", task_config)

task1.start()

In [40]:
band = 'clear_perc'

task_config = {
    'scale': 30,  
    'region': footprint,
    'maxPixels': 5000000000
    }

task2 = ee.batch.Export.image(storm.select(band), f"{stormDartmouthId} - {band}", task_config)

task2.start()